# Preprocessing Wellington Data

**This dataset includes raw data required to calculate carbon emissions, our focus is on studying the decrease in carbon emissions for buses primarily designated for reducing carbon emissions, rather than buses primarily utilized for passenger transportation.**

**While weekend buses tend to be predominantly employed for passenger transportation, it's important to note that weekday buses also serve as a means of passenger transport. However, during weekdays, buses take on an additional role: they are strategically utilized for carbon reduction efforts, particularly during peak hours like 7am-9am and 4pm-6pm.**

### NetBI

The following dimensions were used to generate the datasets

X:
- Actual Running Time
- Actual In-Service KM
- Passenger Km
- Scheduled In-Service Km
- Cancelled Trips
- Sched Running Time per Trip

Y:
- Data
- Route
- Route Variant
- Direction
- Trip Number
- Actual Vehicle Type
- Vehicle Number
- Vehicle Emissions Standard
- Start Minute(Sched)
- Day

### Goals

- Split running time into, hourly run time intervals
- Calculate Average speed of a bus (distance/time) <input type="checkbox" checked> 
- Calculate Average Occupancy (passenger Km/ actual km <input type="checkbox" checked> 
- Calculate Carbon for routes
- Calculate the per person carbon emissions reduction

### Further Goals

- Create Heat Map
- Learn how to upload data to mongoDB
- Integrate heat map into Bean

### Loading Data

In [2]:
import pandas as pd

import pandas as pd

# Load and process DataFrames
jan1_3 = pd.read_csv("Wellington Raw Daily Data/Jan 1-3 2022.csv")
jan4_6 = pd.read_csv("Wellington Raw Daily Data/Jan 4-6 2022.csv")
jan7_9 = pd.read_csv("Wellington Raw Daily Data/Jan 7-9 2022.csv")
jan10_11 = pd.read_csv("Wellington Raw Daily Data/Jan 10-11 2022.csv")
jan12_13 = pd.read_csv("Wellington Raw Daily Data/Jan 12-13 2022.csv")
jan14_15 = pd.read_csv("Wellington Raw Daily Data/Jan 14-15 2022.csv")
jan16_17 = pd.read_csv("Wellington Raw Daily Data/Jan 16-17 2022.csv")
jan18_19 = pd.read_csv("Wellington Raw Daily Data/Jan 18-19 2022.csv")
jan20_21 = pd.read_csv("Wellington Raw Daily Data/Jan 20-21 2022.csv")
jan22_23 = pd.read_csv("Wellington Raw Daily Data/Jan 22-23 2022.csv")
jan24_25 = pd.read_csv("Wellington Raw Daily Data/Jan 24-25 2022.csv")
jan26_27 = pd.read_csv("Wellington Raw Daily Data/Jan 26-27 2022.csv")
jan28_29 = pd.read_csv("Wellington Raw Daily Data/Jan 28-29 2022.csv")
jan30_31 = pd.read_csv("Wellington Raw Daily Data/Jan 30-31 2022.csv")

# List of DataFrames
dataframes = [
    jan1_3, jan4_6, jan7_9, jan10_11, jan12_13,
    jan14_15, jan16_17, jan18_19, jan20_21, jan22_23,
    jan24_25, jan26_27, jan28_29, jan30_31
]

# The bottom row of each data frame must be removed due to it being a total column.
def drop_last_row(df):
    return df.drop(df.tail(1).index)

dataframes = list(map(drop_last_row, dataframes))

# Stacking DataFrames
combined_df = pd.concat(dataframes, ignore_index=True)

# writing and storing processed dataframes as csv
combined_df.to_csv("Wellington Raw Monthly Data/January 2022.csv")

print(combined_df)


               Date Route Route Variant Direction  Trip Number  \
0       01-Jan-2022     1           1_1   Inbound       4000.0   
1       01-Jan-2022     1           1_1   Inbound       4060.0   
2       01-Jan-2022     1           1_1   Inbound       4100.0   
3       01-Jan-2022     1           1_1   Inbound       4140.0   
4       01-Jan-2022     1           1_1   Inbound       4180.0   
...             ...   ...           ...       ...          ...   
100719  31-Jan-2022   WRL         WRL_2  Outbound       1610.0   
100720  31-Jan-2022   WRL         WRL_3   Inbound       1607.0   
100721  31-Jan-2022   WRL         WRL_3   Inbound       1609.0   
100722  31-Jan-2022   WRL         WRL_4  Outbound       1602.0   
100723  31-Jan-2022   WRL         WRL_4  Outbound       1604.0   

       Actual Vehicle Type Vehicle Number Vehicle Emissions Standard  \
0                    ELVDD           3705                   ELECTRIC   
1                    ELVDD           3704                   ELE

In [4]:
#read in sample data from Hamish. Once we have final dataframe, we can repace this one.
full_year_data = pd.read_csv('Wellington Raw Daily Data/Trips_2022-01-01_to_2023-06-30.csv')


In [13]:
full_year_data.head()

,Date,Route,Route Variant,Direction,Trip Number,Vehicle Number,Actual Vehicle Type,Vehicle Emissions Standard,Start Minute (Sched),Actual Running Time,Sched Running Time per Trip,Actual In-Service KM,Scheduled In-Service km,Passenger km,Cancelled Trips,Sighted Stops (%),Day of Week,Speed,Average Occupancy,CO2cost
0,2022-07-01,1,1_1,Inbound,1020,3517,DD,EURO6,23:45,2721,3120,21.239,21.239,172.134,0,0.942308,Friday,28.100110,8.104619,3.052730
1,2022-07-01,1,1_1,Inbound,1040,3713,ELVDD,ELECTRIC,05:45,3052,3480,21.239,21.239,244.018,0,0.980769,Friday,25.052556,11.489147,4.327565
2,2022-07-01,1,1_1,Inbound,1120,3719,ELVDD,ELECTRIC,06:45,3789,4020,21.239,21.239,825.052,0,0.980769,Friday,20.179572,38.846085,14.631980
3,2022-07-01,1,1_1,Inbound,1180,3703,ELVDD,ELECTRIC,07:05,4154,4860,21.239,21.239,538.235,0,0.961538,Friday,18.406452,25.341824,9.545391
4,2022-07-01,1,1_1,Inbound,1240,3705,ELVDD,ELECTRIC,07:25,4483,5580,21.239,21.239,546.413,0,1.000000,Friday,17.055632,25.726870,9.690424


**Bus Speed**

The calculation of bus speed is straightforward and involves the formula where velocity equals the ratio of distance to time ( $ v = \frac{d}{t} $ ).

Since Actual Running Time is in seconds, we divide the time by 3600 to get speed in km her hour.

In [6]:
#Calculate Average speed of a bus (distance/time)
full_year_data['Speed'] = full_year_data['Actual In-Service KM']/(full_year_data['Actual Running Time']/3600)

**Average Occupancy**

 Mean occupancy (per kilometre driven) equals the ratio Passenger km to Actual In-Service KM
 
 Where Passenger km is a unit of measurement representing the transport of 1 passenger over 1 km
 and Actual In-Service KM is the lengh of a trip


In [7]:
full_year_data['Average Occupancy'] = full_year_data['Passenger km']/full_year_data['Actual In-Service KM']

**Average car occupancy**

We use the assumption that on average 1.3 passengers travel in each car. To calculate how many Car km it would take to transport the same number of passengers we divide Passenger km by 1.3

**CO_2 emissions**

CO_2 are taken to be 0.265 kg per Car km. 
To calculate how much CO_2 would be emitted if the bus passengers travelled by a private car we use the following calculation:
$$ CO_2 emissions = \frac{0.265\times Passenger km}{1.3} $$

**Valuation of CO_2 emissions**

The whole-of-government agreed shadow price of carbon ($ per tonne of CO2 equivalent) emissions, in Table 1, is to be used for calculating the economic impact of carbon for transport activities.  This means applying the **central** price path as the default analysis in the economic evaluation of transport proposals and accompanying this with sensitivity analysis based on the low and high price paths. Quoted from here: https://www.nzta.govt.nz/assets/resources/monetised-benefits-and-costs-manual/Monetised-benefits-and-costs-manual.pdf

Table 1: Shadow Price of Carbon (NZ$2022 per tonne of CO2 equivalent)



| Year  | 2023  | 2024  | 2025  | 2026  | 2027  | 2028  | 2029  | 2030  | 2031  | 2032  | 2033  | 2034  |
|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|
| Low   | \$59  | \$65  | \$72  | \$78  | \$85  | \$91  | \$98  | \$104 | \$108 | \$112 | \$116 | \$120 |
| Middle| \$87  | \$97  | \$107 | \$116 | \$126 | \$136 | \$146 | \$155 | \$161 | \$167 | \$174 | \$180 |
| High  | \$171 | \$182 | \$193 | \$203 | \$214 | \$219 | \$224 | \$230 | \$235 | \$241 | \$247 | \$253 |


Therfore the calculation for shadow cost of CO_2 if the same trip was taken by private cars looks as follows:
(dividing by 100 as price is per tonne)

$$ CO_2 shadow \ cost = \frac{0.265\times Passenger km\times \$\ Middle \ price for 2023}{1.3\times1000} $$

Please note that part of our data is from 2022, so technically we should be using 2022 price for that part:
Should your analysis require shadow prices for years prior to 2023, email MBCM@nzta.govt.nz.  

In [12]:
#you can update shadow price here if you want to use it for a different year
shadow_price2023 = 87

full_year_data['CO2cost'] = full_year_data['Passenger km']*0.265*shadow_price2023/(1.3*1000)